# LangChain ChatOpenAI + PydanticOutputParser 예제
이 노트북은 LangChain의 `ChatOpenAI`와 `PydanticOutputParser`를 사용하여 구조화된 출력을 생성하는 방법을 보여줍니다.

In [ ]:
# 필수 라이브러리 설치
#poetry add pydantic

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:7])

gsk_fSX


In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

In [5]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")

In [ ]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)
print(parser.get_format_instructions())

In [7]:
# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [8]:
# 환경변수에서 OpenAI API 키 로드 (실제 사용시 주석 해제)
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [12]:
# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

print(type(output))
pprint(output)

<class '__main__.MovieRecommendation'>
MovieRecommendation(movie_title='스크림(1996)', reason='90년대 감성 물씬 나는 마스크 살인마가 등장하는 메타 공포물. 클래식한 슬래셔 요소에 현대적인 해석을 더해 당대 관객들에게 큰 충격을 줬으며, 오늘날까지도 회자되는 명장면들이 많다.', genre=['공포', '스릴러', '메타퍼블릭', '슬래셔'], estimated_rating=8.5)


In [13]:
# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")

추천 영화: 스크림(1996)
추천 이유: 90년대 감성 물씬 나는 마스크 살인마가 등장하는 메타 공포물. 클래식한 슬래셔 요소에 현대적인 해석을 더해 당대 관객들에게 큰 충격을 줬으며, 오늘날까지도 회자되는 명장면들이 많다.
장르: 공포, 스릴러, 메타퍼블릭, 슬래셔
예상 평점: 8.5/10


## 예상 출력 결과
```
추천 영화: The Silence of the Lambs
추천 이유: 1991년에 개봉한 이 영화는 심리적 공포의 걸작으로 평가받으며, 1990년대를 대표하는 영화 중 하나입니다.
장르: thriller, horror, crime
예상 평점: 8.6/10
```